In [33]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pandas_profiling import ProfileReport

In [4]:
spark = SparkSession.builder.master("local[*]").appName("data quality checks").getOrCreate()


22/08/09 01:03:06 WARN Utils: Your hostname, Piotrs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.10.50.102 instead (on interface en0)
22/08/09 01:03:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/09 01:03:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/09 01:03:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [12]:
df_temp=spark.read.parquet("../hdfs/clean/temperatures")
df_press=spark.read.parquet("../hdfs/clean/pressure")

In [38]:
pd_temp=df_temp.toPandas()
pd_press=df_press.toPandas()

In [35]:
profile = ProfileReport(pd_temp,title="Temperatures profile")
profile.to_file("profiling_reports/temperatures_profile.html")

In [39]:
profile = ProfileReport(pd_press,title="Pressure profile")
profile.to_file("profiling_reports/pressure_profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
df_temp.orderBy([F.col("year").desc(),F.col("month").desc(),F.col("day").desc()]).show(10,False)

+-----+---+------+------+-----+----+----+-----+---------+----+
|month|day|t_morn|t_noon|t_evn|tmax|tmin|tmean|method   |year|
+-----+---+------+------+-----+----+----+-----+---------+----+
|12   |31 |2.5   |3.2   |3.1  |3.4 |2.3 |2.9  |automatic|2017|
|12   |31 |2.5   |3.2   |3.0  |3.4 |2.2 |2.8  |manual   |2017|
|12   |30 |1.4   |1.5   |2.7  |3.2 |1.0 |2.0  |manual   |2017|
|12   |30 |1.4   |1.4   |2.7  |3.1 |1.0 |1.9  |automatic|2017|
|12   |29 |2.9   |2.7   |3.1  |4.8 |2.6 |3.1  |manual   |2017|
|12   |29 |2.9   |2.7   |3.1  |4.8 |2.6 |3.1  |automatic|2017|
|12   |28 |4.3   |4.9   |4.8  |5.5 |3.0 |4.5  |manual   |2017|
|12   |28 |4.3   |4.9   |4.8  |5.5 |3.1 |4.5  |automatic|2017|
|12   |27 |3.0   |3.5   |3.1  |4.4 |2.6 |3.2  |manual   |2017|
|12   |27 |2.9   |3.5   |3.1  |4.4 |2.6 |3.2  |automatic|2017|
+-----+---+------+------+-----+----+----+-----+---------+----+
only showing top 10 rows



Duplicates: we see that there were no repetition of year,month,day if we taken into consideration 'automatic' measuruements taken for years 2013-2017.
Missing values: exact equality means also that there are no missing day in the observation

In [43]:
temp_count_all=df_temp.count()
temp_count_without_auto=df_temp.select("year","month","day").dropDuplicates().count()
from datetime import date
d0=date(2013,1,1)
d1=date(2018,1,1)
temp_count_auto=(d1-d0).days
print(temp_count_all,temp_count_without_auto,temp_count_auto)
assert temp_count_auto+temp_count_without_auto == temp_count_all


97520 95694 1826
